In [173]:
import instances
import importlib
importlib.reload(instances)
Customers = instances.Customers
Points = instances.Points
C, F = instances.C, instances.F
tour = instances.tour

instance_name = "C" + str(len(Customers)-1)

In [174]:
import inputs
As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq = inputs.make_inputs(tour, Customers, Points, F, C)

Preceding constraints are done.
Time-window constarints are done.
Pick-up and delivery constarints are done.
Charge constarints are done.


In [187]:
# Gurobiによって最適解を求める
import formulation
import importlib
importlib.reload(formulation)
num_vars = len(tour)
import time
start = time.time()
P = formulation.solve_primal(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq, instance_name+"P", num_vars)
end = time.time()
print(end-start)

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 166 rows, 155 columns and 402 nonzeros
Model fingerprint: 0x27a7427b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 5e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.010643e+03   6.199997e+07      0s
     132    8.6369240e+02   0.000000e+00   0.000000e+00      0s

Solved in 132 iterations and 0.06 seconds
Optimal objective  8.636924036e+02
0.12475013732910156


In [176]:
threshold = len(tour)//2

former = tour[:threshold]
latter = tour[threshold:]

In [177]:
import separate_coefficient_matrix
As_1, Ac_eq_1, Ac_leq_1, Ar_eq_1, Ar_leq_1, Ap_1, Aq_1, b_eq_1, b_leq_1, op_1, oq_1, \
As_2, Ac_eq_2, Ac_leq_2, Ar_eq_2, Ar_leq_2, Ap_2, Aq_2, b_eq_2, b_leq_2, op_2, oq_2, \
index_P_f_eq, index_P_f_leq, index_P_l_eq, index_P_l_leq \
= separate_coefficient_matrix.separate(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq, threshold)

In [178]:
# 前半と後半に分けてそれぞれを解く

In [179]:
# Gurobiによって最適解を求める
P_f = formulation.solve_primal(As_1, Ac_eq_1, Ac_leq_1, Ar_eq_1, Ar_leq_1, Ap_1, Aq_1, \
                                                       b_eq_1, b_leq_1, op_1, oq_1, instance_name+"P_f", len(former))
print("\n\n")
P_l = formulation.solve_primal(As_2, Ac_eq_2, Ac_leq_2, Ar_eq_2, Ar_leq_2, Ap_2, Aq_2, \
                                                       b_eq_2, b_leq_2, op_2, oq_2, instance_name+"P_l", len(latter))

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 74 rows, 75 columns and 175 nonzeros
Model fingerprint: 0x72ff82ad
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 2e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.277275e+03   3.199998e+07      0s
      49    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 49 iterations and 0.05 seconds
Optimal objective  0.000000000e+00



Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 83 rows, 80 columns

In [180]:
index_P_f_eq = list(map(lambda x:x+As.shape[0], index_P_f_eq))
index_P_l_eq = list(map(lambda x:x+As.shape[0], index_P_l_eq))
index_P_f = list(set(index_P_f_eq) | set(index_P_f_leq))
index_P_l = list(set(index_P_l_eq) | set(index_P_l_leq))

In [181]:
# 初期解の保存
import numpy as np
DStarts = [-var.x for var in P_f.getVars()+P_l.getVars()]

PStarts = [0 for constr in P.getConstrs()]
for i, constr in zip(index_P_f, P_f.getConstrs()):
    PStarts[i] = -constr.Pi
for i, constr in zip(index_P_l, P_l.getConstrs()):
    PStarts[i] = -constr.Pi

In [182]:
# 基底変数、非基底変数の保存
import numpy as np
VBases = [var.VBasis for var in P_f.getVars()+P_l.getVars()]

CBases = [0 for constr in P.getConstrs()]
for i, constr in zip(index_P_f, P_f.getConstrs()):
    CBases[i] = constr.CBasis
for i, constr in zip(index_P_l, P_l.getConstrs()):
    CBases[i] = constr.CBasis

In [183]:
VBases = []
var_names = ['s', 'r', 'c', 'p', 'q']
for i in var_names:
    for j in range(len(former)):
        VBases.append(P_f.getVarByName(i+'['+str(j)+']').VBasis)
    for j in range(len(latter)):
        VBases.append(P_l.getVarByName(i+'['+str(j)+']').VBasis)

In [184]:
CBases = [0 for constr in P.getConstrs()]
ind_pfleq = 0
ind_pfeq = 0
ind_plleq = 0
ind_pleq = 0
for i in range(len(P.getConstrs())):
    if i in index_P_f_leq:
        #print(i)
        CBases[i] = (P_f.getConstrByName('c_leq['+str(ind_pfleq)+']').CBasis)
        ind_pfleq += 1
    if i in index_P_f_eq:
        #print(i)
        CBases[i] = (P_f.getConstrByName('c_eq['+str(ind_pfeq)+']').CBasis)
        ind_pfeq += 1
    if i in index_P_l_leq:
        #print(i)
        CBases[i] = (P_l.getConstrByName('c_leq['+str(ind_plleq)+']').CBasis)
        ind_plleq += 1
    if i in index_P_l_eq:
        #print(i)
        CBases[i] = (P_l.getConstrByName('c_eq['+str(ind_pleq)+']').CBasis)
        ind_pleq += 1

In [185]:
# 基底変数、非基底変数の保存
# 主問題の解をそのまま使う
import numpy as np
CBases_ = [constr.CBasis for constr in P.getConstrs()]
VBases_ = [var.VBasis for var in P.getVars()]

In [188]:
import formulation
import importlib
importlib.reload(formulation)
import time
start = time.time()
D = formulation.solve_dual(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq, \
                                                   instance_name+"D", num_vars, VBases, CBases)
end = time.time()
print(end-start)

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 166 rows, 155 columns and 402 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 5e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5506226e+02   5.935467e+02   0.000000e+00      0s
      16    8.6369240e+02   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.01 seconds
Optimal objective  8.636924036e+02
0.05163407325744629


In [ ]:
for var in P_f.getVars():
    print(var)

In [ ]:
for var in P_l.getVars():
    print(var)

In [ ]:
# 以下で、前半と前半と後半を繋げるための前準備

In [71]:
import numpy as np

In [72]:
padding_1 = np.zeros([As_1.shape[0], As_2.shape[1]])
padding_2= np.zeros([As_2.shape[0], As_1.shape[1]])
As_1_ = np.concatenate([As_1, padding_2], axis=0)
As_2_ = np.concatenate([padding_1, As_2], axis=0)
As_ = np.concatenate([As_1_, As_2_], axis=1)

In [73]:
padding_1 = np.zeros([Ac_eq_1.shape[0], Ac_eq_2.shape[1]])
padding_2= np.zeros([Ac_eq_2.shape[0], Ac_eq_1.shape[1]])
Ac_eq_1_ = np.concatenate([Ac_eq_1, padding_2], axis=0)
Ac_eq_2_ = np.concatenate([padding_1, Ac_eq_2], axis=0)
Ac_eq_ = np.concatenate([Ac_eq_1_, Ac_eq_2_], axis=1)

In [74]:
padding_1 = np.zeros([Ac_leq_1.shape[0], Ac_leq_2.shape[1]])
padding_2= np.zeros([Ac_leq_2.shape[0], Ac_leq_1.shape[1]])
Ac_leq_1_ = np.concatenate([Ac_leq_1, padding_2], axis=0)
Ac_leq_2_ = np.concatenate([padding_1, Ac_leq_2], axis=0)
Ac_leq_ = np.concatenate([Ac_leq_1_, Ac_leq_2_], axis=1)

In [75]:
padding_1 = np.zeros([Ar_eq_1.shape[0], Ar_eq_2.shape[1]])
padding_2= np.zeros([Ar_eq_2.shape[0], Ar_eq_1.shape[1]])
Ar_eq_1_ = np.concatenate([Ar_eq_1, padding_2], axis=0)
Ar_eq_2_ = np.concatenate([padding_1, Ar_eq_2], axis=0)
Ar_eq_ = np.concatenate([Ar_eq_1_, Ar_eq_2_], axis=1)

In [76]:
padding_1 = np.zeros([Ar_leq_1.shape[0], Ar_leq_2.shape[1]])
padding_2= np.zeros([Ar_leq_2.shape[0], Ar_leq_1.shape[1]])
Ar_leq_1_ = np.concatenate([Ar_leq_1, padding_2], axis=0)
Ar_leq_2_ = np.concatenate([padding_1, Ar_leq_2], axis=0)
Ar_leq_ = np.concatenate([Ar_leq_1_, Ar_leq_2_], axis=1)

In [77]:
padding_1 = np.zeros([Ap_1.shape[0], Ap_2.shape[1]])
padding_2= np.zeros([Ap_2.shape[0], Ap_1.shape[1]])
Ap_1_ = np.concatenate([Ap_1, padding_2], axis=0)
Ap_2_ = np.concatenate([padding_1, Ap_2], axis=0)
Ap_ = np.concatenate([Ap_1_, Ap_2_], axis=1)

In [78]:
padding_1 = np.zeros([Aq_1.shape[0], Aq_2.shape[1]])
padding_2= np.zeros([Aq_2.shape[0], Aq_1.shape[1]])
Aq_1_ = np.concatenate([Aq_1, padding_2], axis=0)
Aq_2_ = np.concatenate([padding_1, Aq_2], axis=0)
Aq_ = np.concatenate([Aq_1_, Aq_2_], axis=1)

In [79]:
b_eq_ = np.concatenate([b_eq_1, b_eq_2], axis=0)

In [80]:
b_leq_ = np.concatenate([b_leq_1, b_leq_2], axis=0)

In [81]:
# 前半と後半をつなげたものを解く

In [82]:
import formulation
import importlib
importlib.reload(formulation)
Pf_plus_Pl = formulation.solve_primal(As_, Ac_eq_, Ac_leq_, Ar_eq_, Ar_leq_, Ap_, Aq_, b_eq_, b_leq_, op, oq, \
                                                                  instance_name+"Pf_plus_Pl", len(former)+len(latter))

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 517 rows, 505 columns and 1245 nonzeros
Model fingerprint: 0xd8012ba7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.603384e+04   2.039999e+08      0s
     412    1.3499757e+03   0.000000e+00   0.000000e+00      0s

Solved in 412 iterations and 0.03 seconds
Optimal objective  1.349975705e+03


In [83]:
Pf_plus_Pl.getObjective().getValue()

1349.975705478139

In [84]:
P_f.getObjective().getValue() + P_l.getObjective().getValue()

1349.975705478139

In [ ]:
# 基底解を与えて解く

In [ ]:
c_eq = P_f.getConstrByName("c_eq")

In [ ]:
print(c_eq)

In [ ]:
c = P_l.getConstrs()
print(c)

In [ ]:
VBases__ = []
var_names = ['s', 'r', 'c', 'p', 'q']
for i in var_names:
    for j in range(len(former)):
        VBases__.append(P_f.getVarByName(i+'['+str(j)+']').VBasis)
    for j in range(len(latter)):
        VBases__.append(P_l.getVarByName(i+'['+str(j)+']').VBasis)
#VBases__ = [var.VBasis for var in P_f.getVars()+P_l.getVars()]

CBases__ = [] #[constr.CBasis for constr in P_f.getConstrs()+P_l.getConstrs()]
#constr_names = ["c_leq", "c_eq"]
#for i in constr_names:
for j in range(As_1.shape[0]):
    CBases__.append(P_f.getConstrByName('c_leq['+str(j)+']').CBasis)
for j in range(As_2.shape[0]):
    CBases__.append(P_l.getConstrByName('c_leq['+str(j)+']').CBasis)
for j in range(Ac_eq_1.shape[0]):
    CBases__.append(P_f.getConstrByName('c_eq['+str(j)+']').CBasis)
for j in range(Ac_eq_2.shape[0]):
    CBases__.append(P_l.getConstrByName('c_eq['+str(j)+']').CBasis)

VBases__prime = [var.VBasis for var in Pf_plus_Pl.getVars()]
CBases__prime = [constr.CBasis for constr in Pf_plus_Pl.getConstrs()]

In [ ]:
len(VBases__) == len(VBases__prime)

In [ ]:
for ind, i in enumerate(zip(VBases__, VBases__prime)):
    if i[0]!=i[1]:
        print(ind, "\t", i[0], i[1])

In [ ]:
for i, i_prime in zip(CBases__, CBases__prime):
    if i!=i_prime:
        print(i, i_prime)

In [ ]:
import formulation
import importlib
importlib.reload(formulation)
Pf_plus_Pl = formulation.solve_dual(As_, Ac_eq_, Ac_leq_, Ar_eq_, Ar_leq_, Ap_, Aq_, b_eq_, b_leq_, op, oq, \
                                                                  instance_name+"Pf_plus_Pl", len(former)+len(latter), VBases__, CBases__)

In [ ]:
Pf_plus_Pl.getObjective().getValue()

In [ ]:
P_f.getObjective().getValue() + P_l.getObjective().getValue()

In [57]:
num_vars_1 = b_leq.shape[0]
num_vars_2 = b_eq.shape[0]

In [58]:
import importlib
importlib.reload(formulation)
D_2 = formulation.solve_dual_2(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq, \
                                                   instance_name+"D", num_vars_1, num_vars_2, PStarts, DStarts)

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 505 rows, 528 columns and 1278 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+03]
  Bounds range     [1e+03, 1e+03]
  RHS range        [1e+00, 1e+00]
Crossover log...

       0 DPushes remaining with DInf 1.5635349e+03                 0s

      73 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+00, Dinf 2.9286108e+03      0s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     224    1.3499757e+03   0.000000e+00   2.928611e+03      0s

Solved in 339 iterations and 0.10 seconds
Optimal objective  2.881641047e+03


In [59]:
D.getVars() == D_2.getVars()

False

In [ ]:
for var, var_2 in zip(D.getVars(), D_2.getVars()):
    print(var.varName, var.X, "\t", var_2.X)